### I/O Benchmarks

In [156]:
!rm -rf test*

In [132]:
!mkfile 500m test_500.txt ; mkfile 1024m test_1024.txt ; mkfile 2048m test_2048.txt

In [155]:
!mkfile 100m test_100.txt

In [157]:
import io
import time
import itertools

In [171]:
global benchmarks

In [159]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            benchmarks[method.__name__].append((te - ts) * 1000)
        return result
    return timed

In [160]:
@timeit
def open_io(filename):
    b = bytearray(16) 
    f = io.open(filename, "rb")
    numread = f.readinto(b)
    f.close()
    return f

In [161]:
@timeit
def open_classic(filename):
    with open(filename) as file:
        data = file.read()
    return data

In [173]:
def run_bench(filename, try_cnt=10):
    global benchmarks
    benchmarks = {'open_io':[], "open_classic":[]}
    for i in range(0, try_cnt):
        open_io(filename)
        open_classic(filename)
    return benchmarks

In [174]:
benchmarks = run_bench("test_100.txt")

In [175]:
import plotly.graph_objects as go

x = [i for i in range(0, try_cnt)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=benchmarks['open_io'],
    name = "Open io and byte buffer [100m]",
))
fig.add_trace(go.Scatter(
    x=x,
    y=benchmarks['open_classic'],
    name = "Classic open [100m]",
))

fig.show()